In [1]:
import tensorflow_datasets as tfds
mnist = tfds.load('mnist', with_info=True)
tfds.as_dataframe(mnist[0]['train'].take(4),mnist[1])

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


,image,label
0,,4
1,,1
2,,0
3,,7


In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Flatten, Conv2D,\
                        BatchNormalization, Concatenate, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model

from src.analysis_tools import plot_confusion_matrix, extract_data
from src.cnn_models import basic_cnn, simple_cnn, dense_cnn, coeff_determination, rmse

%load_ext autoreload
%autoreload 2

In [4]:
#load data
import json
stations = ['GrantPoole', 'BrownsBrook','ParkersBrook']

station_name = stations[0]

datafile = f'{station_name}-training.tfrecord.gz'
file_location = f'/datadrive/stream_data/training/{datafile}'
metadata_file = f'/datadrive/stream_data/training/{station_name}-training_metadata.json'
with open(metadata_file) as mfile:
    metadata = json.load(mfile)
ex = metadata['element_spec']['x_data']
ey = metadata['element_spec']['y_data']
element_spec = (tf.TensorSpec(shape=tuple(ex['shape']), dtype=ex['dtype'], name=ex['name']),\
                tf.TensorSpec(shape=tuple(ey['shape']), dtype=ey['dtype'], name=ey['name']))
full_dataset = tf.data.experimental.load(file_location,\
                                          element_spec,\
                                          compression='GZIP')

FileNotFoundError: [Errno 2] No such file or directory: '/datadrive/stream_data/training/GrantPoole-training_metadata.json'

In [44]:
DATASET_SIZE = int(metadata['count'])
prp = 0.7
TRAIN_SIZE = int(prp * DATASET_SIZE)
#VAL_SIZE = int(0 * DATASET_SIZE)
TEST_SIZE = int(1-prp * DATASET_SIZE)
SHUFFLE_BUFFER_SIZE = 100 
BATCH_SIZE = 8 
full_dataset = full_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = full_dataset.take(TRAIN_SIZE)
test_dataset = full_dataset.skip(TRAIN_SIZE)
print(tf.data.experimental.cardinality(train_dataset).numpy())
print(tf.data.experimental.cardinality(test_dataset).numpy())
#val_dataset = test_dataset.skip(TEST_SIZE)
#test_dataset = test_dataset.take(TEST_SIZE)

559
240


In [45]:
'''
f, ax = plt.subplots(1,3, figsize=(20,4))
for i in range(3):
    for images, labels in test_dataset.take(1):
        im = images.numpy()
        label = labels.numpy()
        ax[i].imshow(im)
        ax[i].set_title(label)
'''

'\nf, ax = plt.subplots(1,3, figsize=(20,4))\nfor i in range(3):\n    for images, labels in test_dataset.take(1):\n        im = images.numpy()\n        label = labels.numpy()\n        ax[i].imshow(im)\n        ax[i].set_title(label)\n'

In [48]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

class TestModel(Model):
    def __init__(self):
        super(TestModel, self).__init__()
        self.conv1 = Conv2D(filters=8, kernel_size=(32, 32), activation='relu')
        self.conv2 = Conv2D(filters=4, kernel_size=(16, 16), activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(8, activation='relu')
        self.d2 = Dense(1)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.d1(x)
        return self.d2(x)


model = TestModel()

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.MeanSquaredError()

train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
#train_accuracy = tfa.metrics.RSquare(name='train_R2')
#test_accuracy = tfa.metrics.RSquare(name='test_R2')
train_accuracy = tf.keras.metrics.RootMeanSquaredError(name='train_RMSE')
test_accuracy = tf.keras.metrics.RootMeanSquaredError(name='test_RMSE')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different behavior
    # during training versus inference (e.g. Dropout)
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [49]:
EPOCHS = 2 
BATCH_SIZE = 16 
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_dataset.batch(BATCH_SIZE):
        train_step(images, labels)
        
    for test_images, test_labels in test_dataset.batch(BATCH_SIZE):
        test_step(test_images, test_labels)
    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f},\
                Test Loss: {:.2f}, Test Accuracy: {:.2f}'
    #template = 'Epoch {}, Loss: {}, Test Loss: {}'
    print(f'')
    print(template.format(epoch + 1,
                        train_loss.result(),\
                        test_loss.result(),\
                        train_accuracy.result(),\
                        test_accuracy.result()))


Epoch 1, Loss: 0.87, Accuracy: 0.93,                Test Loss: 0.93, Test Accuracy: 0.97

Epoch 2, Loss: 0.50, Accuracy: 0.53,                Test Loss: 0.71, Test Accuracy: 0.73


In [ ]:
#build model and fit

#BATCH_SIZE = 8 
#train_dataset = train_dataset.batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)

In [9]:
input_shape = tuple(ex['shape'])
output_shape = 1

In [10]:
output_shape

1

In [16]:
model = basic_cnn(input_shape=input_shape,\
                  output_shape=output_shape,\
                  final_activation='relu')
#model = simple_cnn(input_shape=input_shape, output_shape=output_shape)
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 960, 1280, 3)]    0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 960, 1280, 16)     448       
_________________________________________________________________
batch_normalization_22 (Batc (None, 960, 1280, 16)     64        
_________________________________________________________________
activation_18 (Activation)   (None, 960, 1280, 16)     0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 960, 1280, 16)     64        
_________________________________________________________________
dense_5 (Dense)              (None, 960, 1280, 10)     170       
_________________________________________________________________
output_layer (Dense)         (None, 960, 1280, 1)     

In [20]:
opt = Adam(lr=0.0001)
epochs = 10 
model.compile(loss=rmse, optimizer=opt, metrics=[coeff_determination])

In [21]:
model.fit(train_dataset, epochs)

ValueError: `y` argument is not supported when using dataset as input.

In [ ]:
shift = 7 
r_range = 5 
batch_size = 8 
brightness_range=(0.8,1)
datagen = ImageDataGenerator(rescale=1./255,\
                             rotation_range=r_range,\
                             width_shift_range=shift,\
                             height_shift_range=shift,\
                             brightness_range=brightness_range)
datagen.fit(x_train, augment=True, seed=1234)

In [ ]:
z = next(datagen.flow(x_train,y_train, batch_size))
f, ax = plt.subplots(1,batch_size, figsize=(20,10))
for i in range(batch_size):
    ax[i].imshow(z[0][i])
    ax[i].set_axis_off()

In [19]:
opt = Adam(lr=0.0001)
epochs = 10 
model.compile(loss=rmse, optimizer=opt, metrics=[coeff_determination])

acc_hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size), \
                               steps_per_epoch=len(x_train)/batch_size, \
                               epochs=epochs, \
                               class_weight=class_weights,\
                               validation_data=(x_test, y_test))

NameError: name 'datagen' is not defined

In [ ]:
if hist:
    hist_add = acc_hist.history
    for k in hist.keys():
        hist[k].extend(hist_add[k])
else:
    hist = acc_hist.history

In [ ]:
#hist = acc_hist.history
f, ax = plt.subplots(2,1,figsize=(10,8))
for k in hist.keys():
    if 'loss' in k:
        ax[0].plot(hist[k], label=k)
    else:
        ax[1].plot(hist[k], label=k)
ylabels = ['Loss', 'Accuracy']
for i in range(2):
    ax[i].legend()
    ax[i].grid(alpha=0.5)
    ax[i].set_ylabel(ylabels[i])
    ax[i].set_xlabel('Epoch')
f.savefig('/content/output/accuracy.png', dpi=100, bbox_inches='tight', pad_inches=0)

In [ ]:
y_hat = model.predict(x_test)
ii = 7 
jj = 7
sample_i = np.random.choice(len(x_test), ii*jj, replace=False).reshape((ii,jj))
label = ['low flow','mid flow','high flow']
color = ['orange', 'darkblue']
f,ax = plt.subplots(ii, jj, figsize=(18,30))
for i in range(ii):
    for j in range(jj):
        s = sample_i[i][j]
        ax[i][j].imshow(x_test[s])
        ax[i][j].set_axis_off()
        ax[i][j].set_title('%s\nFlowrate: %s\nTrue:%s\nPredicted:%s'%(dates_test[s],y_test_flow[s],\
                                             label[np.argmax(y_test[s])],label[np.argmax(y_hat[s])]),\
                                                         color=color[np.argmax(y_test[s])==np.argmax(y_hat[s])])
f.savefig('/content/output/validation.png', dpi=100, bbox_inches='tight', pad_inches=0)

In [ ]:
cm = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(y_hat,axis=1))
plot_confusion_matrix(cm, classes=label, outname='/content/output/conf_mat.png')

In [ ]:
model_json = model.to_json()
with open("/content/output/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/output/model.h5")
print("Saved model to disk")
plot_model(model, to_file='/content/output/model_graph.png')